In [97]:
#importo las librerias necesarias

import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [98]:
df = pd.read_csv('Datasets/dataset_limpio.csv')

In [99]:
#chequeo que se importaron bien los datos
df.head()

,id,title,genres,overview,tagline,spoken_languages,decade
0,862,Toy Story,"['Animation', 'Comedy', 'Family']","Led by Woody, Andy's toys live happily in his ...",,['English'],1990
1,8844,Jumanji,"['Adventure', 'Fantasy', 'Family']",When siblings Judy and Peter discover an encha...,Roll the dice and unleash the excitement!,"['English', 'Français']",1990
2,15602,Grumpier Old Men,"['Romance', 'Comedy']",A family wedding reignites the ancient feud be...,Still Yelling. Still Fighting. Still Ready for...,['English'],1990
3,31357,Waiting to Exhale,"['Comedy', 'Drama', 'Romance']","Cheated on, mistreated and stepped on, the wom...",Friends are the people who let you be yourself...,['English'],1990
4,11862,Father of the Bride Part II,['Comedy'],Just when George Banks has recovered from his ...,Just When His World Is Back To Normal... He's ...,['English'],1990


In [86]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42196 entries, 0 to 42195
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   id                42196 non-null  object
 1   title             42196 non-null  object
 2   genres            42196 non-null  object
 3   overview          42196 non-null  object
 4   tagline           42196 non-null  object
 5   spoken_languages  42196 non-null  object
 6   decade            42196 non-null  object
dtypes: object(7)
memory usage: 2.3+ MB


In [87]:
df.isnull().sum()

id                  0
title               0
genres              0
overview            0
tagline             0
spoken_languages    0
decade              0
dtype: int64

la idea principal es crear dos matrices, una para 'overview' y 'tagline' calculando la simiitud usando la matriz TF-IDF ya que este metodo pondera la importancia de cada palabra en relacion a todo el texto, lo que captura mejor las palabras claves o terminos especificos. 
para 'genres', 'decade' y 'spoken_languages' usar una matriz de conteo ya que esta simplemente cuenta la frecuencia de cada palabra sin tener en cuenta el resto del texto, considerando que estos datos son más cercanos a un metadato que a datos de lenguaje como si ocurre en 'overview' y 'tagline'

vamos a empezar con la matriz TF-IDF

In [100]:
#eliminación de caracteres especiales y conversión a minúsculas:
df['overview'] = df['overview'].str.replace('[^\w\s]', '').str.lower().str.replace(",", "").str.replace(".", "").str.replace("!", "")
df['tagline'] = df['tagline'].str.replace("[^\w\s]", "").str.lower().str.replace(",", "").str.replace(".", "").str.replace("!", "")

In [101]:
#eliminación de palabras vacías (stopwords) y lematización:
nltk.download('stopwords')
nltk.download('wordnet')

#nltk.download('omw-1.4')
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

df['overview'] = df['overview'].apply(lambda x: ' '.join([lemmatizer.lemmatize(word) for word in x.split() if word not in stop_words]))
df['tagline'] = df['tagline'].apply(lambda x: ' '.join([lemmatizer.lemmatize(word) for word in x.split() if word not in stop_words]))

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/ssanjua/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/ssanjua/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


KeyboardInterrupt: 

In [ ]:
df['overview'].iloc[0]

"led woody andy's toy live happily room andy's birthday brings buzz lightyear onto scene afraid losing place andy's heart woody plot buzz circumstance separate buzz woody owner duo eventually learns put aside difference"

In [ ]:
df['tagline'].iloc[1]

'roll dice unleash excitement'

In [102]:
text_data = df[['overview', 'tagline']]
#combinamos los campos en un solo texto
df['texto_combinado'] = text_data['overview'] + ' ' + text_data['tagline']

In [103]:
#eliminamos las columnas ya que no las necesitaremos mas
df = df.drop('overview', axis=1)
df = df.drop('tagline', axis=1)

In [104]:
df.head()

,id,title,genres,spoken_languages,decade,texto_combinado
0,862,Toy Story,"['Animation', 'Comedy', 'Family']",['English'],1990,led by woody andy's toys live happily in his r...
1,8844,Jumanji,"['Adventure', 'Fantasy', 'Family']","['English', 'Français']",1990,when siblings judy and peter discover an encha...
2,15602,Grumpier Old Men,"['Romance', 'Comedy']",['English'],1990,a family wedding reignites the ancient feud be...
3,31357,Waiting to Exhale,"['Comedy', 'Drama', 'Romance']",['English'],1990,cheated on mistreated and stepped on the women...
4,11862,Father of the Bride Part II,['Comedy'],['English'],1990,just when george banks has recovered from his ...


In [105]:
# Crear el vectorizador TF-IDF. lo limito a 50000 para que no sea tan grande
vectorizer = TfidfVectorizer(analyzer='word', stop_words='english', max_features=50000)

# Aplicar el vectorizador a los datos de texto
tfidf_matrix = vectorizer.fit_transform(df['texto_combinado'])

In [106]:
tfidf_matrix.shape

(42196, 50000)

In [107]:
#similaritud del coseno: 
coseno_sim_text = cosine_similarity(tfidf_matrix)

#toma entre 35s a 50s

COUNT MATRIX

In [109]:
#quito los espacios en los generos formados por mas de una silaba
df['genres'] = df['genres'].str.replace(" ", "")

In [110]:
#duplico 'genres' para darle mas importancia y las uno en un campo
df['combined_features'] = df['genres'] + df['genres'] + df['decade'] + df['spoken_languages']

In [111]:
df['combined_features'][7]

"['Action','Adventure','Drama','Family']['Action','Adventure','Drama','Family']1990['English', 'Deutsch']"

In [112]:
#elimino las columnas que ya no me sirven
df = df.drop('genres', axis=1)
df = df.drop('decade', axis=1)
df = df.drop('id', axis=1)
df = df.drop('spoken_languages', axis=1)

In [113]:
df['combined_features'] = df['combined_features'].str.replace("'", " ").str.replace("[", " ").str.replace("]", " ").str.replace(",", " ").str.lower()

In [114]:
df.head(5)

,title,texto_combinado,combined_features
0,Toy Story,led by woody andy's toys live happily in his r...,animation comedy family animation c...
1,Jumanji,when siblings judy and peter discover an encha...,adventure fantasy family adventure ...
2,Grumpier Old Men,a family wedding reignites the ancient feud be...,romance comedy romance comedy 1990 ...
3,Waiting to Exhale,cheated on mistreated and stepped on the women...,comedy drama romance comedy drama ...
4,Father of the Bride Part II,just when george banks has recovered from his ...,comedy comedy 1990 english


In [115]:
#Vectorización de texto
vectorizer = CountVectorizer(stop_words='english')
feature_matrix = vectorizer.fit_transform(df['combined_features'])

In [116]:
feature_matrix.shape

(42196, 246)

In [117]:
coseno_sim_features = cosine_similarity(feature_matrix)

#tarda mas de 1min y medio

In [118]:
# Combina las similitudes utilizando algún método (por ejemplo, suma ponderada)
combined_similarity = 0.6 * coseno_sim_text + 0.4 * coseno_sim_features

#tarda mas de 5min

In [119]:
def get_recommendations(title):
    titles = df['title']
    indices = pd.Series(df.index, index=df['title'])  
    idx = indices[title]
    sim_scores = list(enumerate(combined_similarity[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:6]  # Obtenemos las 20 películas más similares
    movie_indices = [i[0] for i in sim_scores]
    
    return df['title'].iloc[movie_indices]

In [120]:
get_recommendations('Toy Story')

2961                Toy Story 2
14764               Toy Story 3
23200                 Small Fry
20859    Bartok the Magnificent
726               A Close Shave
Name: title, dtype: object

In [121]:
get_recommendations('Minions')

20145    Despicable Me 2
5271     Stuart Little 2
40892    Despicable Me 3
18828     Wreck-It Ralph
32789          Dino Time
Name: title, dtype: object

In [122]:
get_recommendations('Avatar')

6320     Lara Croft Tomb Raider: The Cradle of Life
25040                                Thor: Ragnarok
9904                                 Fantastic Four
14579                                  Solomon Kane
5775                               Heavy Metal 2000
Name: title, dtype: object

In [123]:
get_recommendations('Titanic')

2045      Return to Paradise
342              Bitter Moon
652                     Fear
9255    When Will I Be Loved
1418                   Crash
Name: title, dtype: object

In [124]:
get_recommendations('Superman')

2502                          Superman II
10733                    Superman Returns
2504     Superman IV: The Quest for Peace
30221                Atom Man vs Superman
20058                        Man of Steel
Name: title, dtype: object

In [125]:
df.shape

(42196, 3)

el modelo funciona bastante bien hasta acá, pero demora demasiado en ejecutarse por lo que desde ya voy a abandonar la idea de aplicar un filtro para que considere las puntuaciones y la popularidad porque va a seguir sumando carga al modelo.

para poder deployar y hacerlo funcionar en tiempo razonable voy a tener que reducir el tamaño de mi set de prueba, intentando seguir con las dos matrices pero entendiendo que lo que más tiempo le lleva al modelo es encontrarla similaritud entre ambas.

In [134]:
#creo un nuevo dataset mas pequeño
small_df = df.sample(n=8000, random_state=42)

#le reseteo el indice para evitar problemas
small_df=small_df.reset_index(drop=True)

In [135]:
small_df

,title,texto_combinado,combined_features
0,The Business of Fancydancing,seymour polatkin is a successful gay indian po...,music drama music drama 2000 english
1,Dollman,brick bardo is a traveller from outer space wh...,action comedy crime sciencefiction ...
2,The Dukes,the dukesa doo wop group were on top of the wo...,comedy crime drama music comedy c...
3,Dragon Wars: D-War,based on the korean legend unknown creatures w...,fantasy drama horror action thriller...
4,Tom and Huck,a mischievous young boy tom sawyer witnesses a...,action adventure drama family actio...
...,...,...,...
7995,Surviving Sid,sid the sloth takes a school of children out o...,animation animation 2000 english
7996,The Devil's Ground,while traveling from california to bangor thro...,horror thriller horror thriller 2000...
7997,Keep Your Right Up,this film is made up several sketches in which...,comedy drama comedy drama 1980 fran...
7998,Mac,niccolo (mac) vitelli is the eldest of three b...,drama drama 1990 english


In [140]:
# Crear el vectorizador TF-IDF. lo limito a 50000 para que no sea tan grande
vectorizerSML = TfidfVectorizer(analyzer='word', stop_words='english', max_features=50000)

# Aplicar el vectorizador a los datos de texto
tfidf_matrixSML = vectorizerSML.fit_transform(small_df['texto_combinado'])

tfidf_matrixSML.shape

(8000, 32877)

In [139]:
#similaritud del coseno del TF-IDF: 
coseno_sim_textdSML = cosine_similarity(tfidf_matrixSML)

In [142]:
#conteo de vectores con los features
vectorizerSML_features = CountVectorizer(stop_words='english')
feature_matrixSML = vectorizerSML_features.fit_transform(small_df['combined_features'])

feature_matrixSML.shape

(8000, 167)

In [143]:
#similaritud del coseno
coseno_sim_featuresSML = cosine_similarity(feature_matrixSML)

In [145]:
# Combina las similitudes utilizando algún método (por ejemplo, suma ponderada)
combined_similaritySML = 0.6 * coseno_sim_textdSML + 0.4 * coseno_sim_featuresSML

#demora menos de 2s

In [157]:
def get_recommendationsSML(title):
    titles = small_df['title']
    indices = pd.Series(small_df.index, index=small_df['title'])  
    if title not in indices:
        return "La película no está en el dataset reducido"
    idx = indices[title]
    sim_scores = list(enumerate(combined_similaritySML[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:6]  # Obtenemos las 5 películas más similares
    movie_indices = [i[0] for i in sim_scores]
    
    return df['title'].iloc[movie_indices]

In [155]:
get_recommendationsSML('The Dukes')

6988               Love at Large
563            Little Big League
7384    Smiles of a Summer Night
7141                     Godsend
2982        The James Dean Story
Name: title, dtype: object

In [158]:
get_recommendationsSML('Toy Story')

'La película no está en el dataset reducido'

In [169]:
#definicion final para API:
def recomendacion(title:str):
    indices = small_df[small_df['title'] == title]
    if indices.empty:
        return "La película no está en el dataset reducido"

    idx = indices.index[0]
    vectorizerTF = TfidfVectorizer(analyzer='word', stop_words='english')
    tfidf_matrixTF = vectorizerTF.fit_transform(small_df['texto_combinado'])
    coseno_sim_text = cosine_similarity(tfidf_matrixTF)

    vectorizer_features = CountVectorizer(stop_words='english')
    feature_matrix = vectorizer_features.fit_transform(small_df['combined_features'])
    coseno_sim_features = cosine_similarity(feature_matrix)

    combined_similarity = 0.6 * coseno_sim_text + 0.4 * coseno_sim_features

    sim_scores = list(enumerate(combined_similarity[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:6]  # Obtenemos las 5 películas más similares
    movie_indices = [i[0] for i in sim_scores]
    movie_titles = [small_df['title'].iloc[i].title() for i in movie_indices]

    
    return {'lista recomendada': movie_titles} 

In [170]:
recomendacion('Minion')

'La película no está en el dataset reducido'

In [171]:
recomendacion('Toy Story')

'La película no está en el dataset reducido'

In [172]:
recomendacion('The Dukes')

{'lista recomendada': ["Mini'S First Time",
  'Girltrash: All Night Long',
  'Taking Woodstock',
  'Duets',
  'Little Fish, Strange Pond']}

In [173]:
#small_df.to_csv('Datasets/dataset_reducido.csv', index=False)

In [52]:
small_df = pd.read_csv('Datasets/dataset_reducido.csv')

In [174]:
def recomendacion2(title:str):
    '''Ingresas un nombre de pelicula y te recomienda las similares en una lista'''
    #small_df = pd.read_csv('../Datasets/dataset_reducido.csv')
    indices = pd.Series(df.index, index=df['title'])  
    if title not in indices:
        return "La película no está en el dataset reducido"
    idx = indices[title]
    vectorizer = TfidfVectorizer(analyzer='word', stop_words='english')
    tfidf_matrix = vectorizer.fit_transform(small_df['texto_combinado'])
    coseno_sim_text = cosine_similarity(tfidf_matrix)
    vectorizer_features = CountVectorizer(stop_words='english')
    feature_matrix = vectorizer_features.fit_transform(small_df['combined_features'])
    coseno_sim_features = cosine_similarity(feature_matrix)
    combined_similarity = 0.6 * coseno_sim_text + 0.4 * coseno_sim_features

    sim_scores = list(enumerate(combined_similarity[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:6]  # Obtenemos las 5 películas más similares
    movie_indices = [i[0] for i in sim_scores]
    recommendations=list(small_df['title'].iloc[movie_indices].str.title())
    
    return {'lista recomendada': recommendations} 

In [175]:
recomendacion2('Toy Story')

{'lista recomendada': ['Hendrix',
  'Masked And Anonymous',
  'Passing Strange',
  'Madame Sousatzka',
  'Winter Solstice']}

In [55]:
small_df

,title,texto_combinado,combined_features
0,The Business of Fancydancing,seymour polatkin successful gay indian poet sp...,music drama music drama 2000 english
1,Dollman,brick bardo traveller outer space forced land ...,action comedy crime sciencefiction ...
2,The Dukes,dukesa doo wop group top world 17 struggling s...,comedy crime drama music comedy c...
3,Dragon Wars: D-War,based korean legend unknown creature return de...,fantasy drama horror action thriller...
4,Tom and Huck,mischievous young boy tom sawyer witness murde...,action adventure drama family actio...
...,...,...,...
9995,Slaughterhouse,owner slaughterhouse facing foreclosure instru...,horror horror 1980 english
9996,Mother's Heart,lorenz three young child victim medium wish tu...,drama drama 1960 italiano
9997,Antisocial,five university friend gather house party ring...,horror thriller horror thriller 2010...
9998,Shadows of the Dead,group teenager try escape creature life among ...,horror horror 2010 english


In [60]:
small_df['text'] = small_df['texto_combinado'] + small_df['combined_features']

In [63]:
tiny_df = small_df[['title', 'text']]

In [82]:
tiny_df.to_csv('Datasets/dataset_tiny.csv', index=False)

In [76]:
def recomendacionTINY(titulo:str):
    '''Ingresas un nombre de pelicula y te recomienda las similares en una lista'''
    indices = tiny_df[tiny_df['title'] == titulo]
    if indices.empty:
        return "La película no está en el dataset reducido"
    idx = indices.index[0]
    vectorizer = TfidfVectorizer(analyzer='word', stop_words='english')
    tfidf_matrix = vectorizer.fit_transform(tiny_df['text'])
    coseno_sim = cosine_similarity(tfidf_matrix)
    sim_scores = list(enumerate(coseno_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:6]  # Obtenemos las 5 películas más similares
    movie_indices = [i[0] for i in sim_scores]
    recommendations = list(tiny_df['title'].iloc[movie_indices].str.title())
    
    return {'lista recomendada': recommendations} 

In [78]:
recomendacionTINY('The Dukes')

{'lista recomendada': ['Beautiful Vera',
  'Get Yourself A College Girl',
  'God Save The King',
  'I Want You',
  'Rock On 2']}

La siguiente etapa implica la construcción de una matriz de similitud entre las películas. Esta matriz nos ayudará a calcular la similitud entre películas y así generar recomendaciones basadas en películas similares.

In [ ]:
@app.get('/recomendacion/{titulo}')
def recomendacion(titulo:str):
    '''Ingresas un nombre de pelicula y te recomienda las similares en una lista'''
    indices = small_df[small_df['title'] == titulo]
    if indices.empty:
        return "La película no está en el dataset reducido"
    idx = indices.index[0]
    vectorizer = TfidfVectorizer(analyzer='word', stop_words='english')
    tfidf_matrix = vectorizer.fit_transform(small_df['texto_combinado'])
    coseno_sim_text = cosine_similarity(tfidf_matrix)
    vectorizer_features = CountVectorizer(stop_words='english')
    feature_matrix = vectorizer_features.fit_transform(small_df['combined_features'])
    coseno_sim_features = cosine_similarity(feature_matrix)
    combined_similarity = 0.6 * coseno_sim_text + 0.4 * coseno_sim_features

    sim_scores = list(enumerate(combined_similarity[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:6]  # Obtenemos las 5 películas más similares
    movie_indices = [i[0] for i in sim_scores]
    recommendations=list(small_df['title'].iloc[movie_indices].str.title())
    
    return {'lista recomendada': recommendations} 